In [1]:
import pandas as pd
import pandas as np
import re

In [2]:
def print_distribution(df, col):
    positive = np.count_nonzero(df[col] == 1)
    negative = np.count_nonzero(df[col] == 0)
    print("- Positive:", positive)
    print("- Negative:", negative)
    print("- Verhältnis:", negative / positive)

### Neue Daten einlesen & vereinheitlichen - hate-speech-and-offensive-language

In [15]:
df_offensive_raw = pd.read_csv("hate-speech-and-offensive-language/ORIGINAL_DATA.csv", index_col=0, on_bad_lines='warn')
df_offensive_raw.head(10)

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [16]:
df_offensive_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24783 entries, 0 to 25296
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   count               24783 non-null  int64 
 1   hate_speech         24783 non-null  int64 
 2   offensive_language  24783 non-null  int64 
 3   neither             24783 non-null  int64 
 4   class               24783 non-null  int64 
 5   tweet               24783 non-null  object
dtypes: int64(5), object(1)
memory usage: 1.3+ MB


#### Label erstellen

In [17]:
df_offensive_raw['label'] = df_offensive_raw['class'].replace({
    0:1,
    1:0,
    2:0,
})
df_offensive_raw.head()

,count,hate_speech,offensive_language,neither,class,tweet,label
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0


In [18]:
df_offensive_raw.label.value_counts()

0    23353
1     1430
Name: label, dtype: int64

#### Userhandles anonymisieren

In [19]:
df_offensive_raw['userhandles'] = [re.findall(r'@\w+', x) if re.findall(r'@\w+', x) else [] for x in df_offensive_raw['tweet']]
df_offensive_raw['tweet'] = df_offensive_raw['tweet'].str.replace(r'@\w+', '@user', regex=True)

In [20]:
df_offensive_raw.head()

,count,hate_speech,offensive_language,neither,class,tweet,label,userhandles
0,3,0,0,3,2,!!! RT @user: As a woman you shouldn't complai...,0,[@mayasolovely]
1,3,0,3,0,1,!!!!! RT @user: boy dats cold...tyga dwn bad f...,0,[@mleew17]
2,3,0,3,0,1,!!!!!!! RT @user Dawg!!!! RT @user: You ever f...,0,"[@UrKindOfBrand, @80sbaby4life]"
3,3,0,2,1,1,!!!!!!!!! RT @user: @user she look like a tranny,0,"[@C_G_Anderson, @viva_based]"
4,6,0,6,0,1,!!!!!!!!!!!!! RT @user: The shit you hear abou...,0,[@ShenikaRoberts]


#### CSV speichern

In [21]:
df_offensive_new = df_offensive_raw[['tweet', 'label']]
df_offensive_new.to_csv("hate-speech-and-offensive-language/new_data.csv")

### Neue Daten einlesen & vereinheitlichen - detecting-offensive-language-in-tweets

In [22]:
df_detecting_raw = pd.read_csv("detecting-offensive-language-in-tweets/ORIGINAL_DATA.csv", index_col=0, on_bad_lines='warn', delimiter='\t', names=['id', 'tweet', 'category'])
df_detecting_raw.head(10)

,tweet,category
id,,
572340447797903360,RT @colonelkickhead: Another bloody instant re...,none
560934139403436032,@azzamalirhabi @JihadiA8 This video of the Pes...,none
575597326519042050,Oh really? No more instant restaurants? THAT'S...,none
552487055553757187,RT @Benfrancisallen: It hasn't been a good few...,none
563306298788151300,RT @NoToFeminism: I don’t need femisnsn becaus...,none
562782575656435717,@MariachiMacabre 19% is not the vast majority,none
571776252656287744,@DianH4 @ExposeFalsehood And it is Muslims who...,racism
567015045561712641,@truaemusic @mattybboi83 @Number10gov Capital ...,racism
565363767090085896,RT @fruitondabottom: #FeminismisEQUALITYwhen M...,sexism


In [23]:
df_detecting_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16050 entries, 572340447797903360 to 564921657240735744
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet     16050 non-null  object
 1   category  16050 non-null  object
dtypes: object(2)
memory usage: 376.2+ KB


#### Label erstellen

In [24]:
df_detecting_raw['category'] = df_detecting_raw['category'].str.strip()

df_detecting_raw['label'] = df_detecting_raw['category'].replace({
    "racism":1,
    "sexism":1,
    "none":0,
})
df_detecting_raw.head(10)

,tweet,category,label
id,,,
572340447797903360,RT @colonelkickhead: Another bloody instant re...,none,0
560934139403436032,@azzamalirhabi @JihadiA8 This video of the Pes...,none,0
575597326519042050,Oh really? No more instant restaurants? THAT'S...,none,0
552487055553757187,RT @Benfrancisallen: It hasn't been a good few...,none,0
563306298788151300,RT @NoToFeminism: I don’t need femisnsn becaus...,none,0
562782575656435717,@MariachiMacabre 19% is not the vast majority,none,0
571776252656287744,@DianH4 @ExposeFalsehood And it is Muslims who...,racism,1
567015045561712641,@truaemusic @mattybboi83 @Number10gov Capital ...,racism,1
565363767090085896,RT @fruitondabottom: #FeminismisEQUALITYwhen M...,sexism,1


In [25]:
df_detecting_raw.label.value_counts()

0    11008
1     5042
Name: label, dtype: int64

#### Userhandles anonymisieren

In [26]:
df_detecting_raw['userhandles'] = [re.findall(r'@\w+', x) if re.findall(r'@\w+', x) else [] for x in df_detecting_raw['tweet']]
df_detecting_raw['tweet'] = df_detecting_raw['tweet'].str.replace(r'@\w+', '@user', regex=True)
df_detecting_raw.head(100)

,tweet,category,label,userhandles
id,,,,
572340447797903360,RT @user: Another bloody instant restaurant we...,none,0,[@colonelkickhead]
560934139403436032,@user @user This video of the Peshmerga decima...,none,0,"[@azzamalirhabi, @JihadiA8]"
575597326519042050,Oh really? No more instant restaurants? THAT'S...,none,0,[]
552487055553757187,RT @user: It hasn't been a good few weeks for ...,none,0,[@Benfrancisallen]
563306298788151300,RT @user: I don’t need femisnsn because men ca...,none,0,[@NoToFeminism]
...,...,...,...,...
552245992721252352,RT @user: #ISIS #Islam PC Puzzle: Converting t...,racism,1,[@PeterTownsend7]
575950853644775424,RT @user: Mmm maple bacon is the bomb #mkr,none,0,[@sl473]
575506339029352449,@user @user Lol. You think there are no probs ...,racism,1,"[@BabaKristian, @MuftahOrg]"


#### CSV speichern

In [27]:
df_detecting_new = df_detecting_raw[['tweet', 'label']]
df_detecting_new.to_csv("detecting-offensive-language-in-tweets/new_data.csv")

### Neue Daten einlesen & vereinheitlichen - measuring-hate-speech

In [28]:
df_measuring_raw = pd.read_parquet("measuring-hate-speech/ORIGINAL_DATA.parquet")
df_measuring_raw.head()

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,annotator_religion_hindu,annotator_religion_jewish,annotator_religion_mormon,annotator_religion_muslim,annotator_religion_nothing,annotator_religion_other,annotator_sexuality_bisexual,annotator_sexuality_gay,annotator_sexuality_straight,annotator_sexuality_other
0,47777,10873,3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,39773,2790,2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,47101,3379,3,4.0,4.0,4.0,4.0,4.0,4.0,0.0,...,False,False,False,False,True,False,False,False,True,False
3,43625,7365,3,2.0,3.0,2.0,1.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
4,12538,488,0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,False,False,False,False,False,False,False,False,True,False


In [29]:
df_measuring_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135556 entries, 0 to 135555
Columns: 131 entries, comment_id to annotator_sexuality_other
dtypes: bool(103), float64(19), int32(2), int8(1), object(6)
memory usage: 40.3+ MB


In [30]:
df_measuring_raw['label'] = (df_measuring_raw['hatespeech'] == 2.0).astype(int)

In [31]:
df_measuring_raw.label.value_counts()

0    89535
1    46021
Name: label, dtype: int64

In [32]:
df_measuring_raw = df_measuring_raw[['text', 'label']]
df_measuring_raw.rename(columns={'text':'tweet'}, inplace=True)
df_measuring_raw

,tweet,label
0,Yes indeed. She sort of reminds me of the elde...,0
1,The trans women reading this tweet right now i...,0
2,Question: These 4 broads who criticize America...,1
3,It is about time for all illegals to go back t...,0
4,For starters bend over the one in pink and kic...,1
...,...,...
135551,عاجل سماحة #السيد_عبدالملك_بدرالدين_الحوثي نص...,0
135552,Millions of #Yemen-is participated in mass ral...,0
135553,@AbeShinzo @realDonaldTrump @shinzoabe 独裁者は行きま...,0
135554,Millions of #Yemen-is participated in mass ral...,0


#### Userhandles anonymisieren

In [33]:
df_measuring_raw['userhandles'] = [re.findall(r'@\w+', x) if re.findall(r'@\w+', x) else [] for x in df_measuring_raw['tweet']]
df_measuring_raw['tweet'] = df_measuring_raw['tweet'].str.replace(r'@\w+', '@user', regex=True)
df_measuring_raw.head()

,tweet,label,userhandles
0,Yes indeed. She sort of reminds me of the elde...,0,[]
1,The trans women reading this tweet right now i...,0,[]
2,Question: These 4 broads who criticize America...,1,[]
3,It is about time for all illegals to go back t...,0,[]
4,For starters bend over the one in pink and kic...,1,[]


In [34]:
df_measuring_raw.userhandles.value_counts()

[]                                                                  119352
[@Legionary1488, @JomavaesJos]                                         799
[@Cam_Coles]                                                           793
[@JihadistJoe]                                                         772
[@iamkrause]                                                           767
                                                                     ...  
[@SWRTxxx, @CicisSWPromo, @Lewd_Dudes, @MoondustRT, @QueerBabes]         1
[@David_Mahfood]                                                         1
[@StemCell]                                                              1
[@ResistProfessor, @PinkIguana11]                                        1
[@rextrekr, @peterdaou]                                                  1
Name: userhandles, Length: 4820, dtype: int64

#### CSV speichern

In [35]:
df_measuring_new = df_measuring_raw[['tweet', 'label']]
df_measuring_new.to_csv("measuring-hate-speech/new_data.csv")

### Neue Daten einlesen & vereinheitlichen - Benchmark GAB/REDDIT

In [36]:
# todo

### Datensätze zusammenführen

In [11]:
df_measuring_new = pd.read_csv("measuring-hate-speech/new_data.csv", index_col=0)
df_offensive_new = pd.read_csv("hate-speech-and-offensive-language/new_data.csv", index_col=0)
df_detecting_new = pd.read_csv("detecting-offensive-language-in-tweets/new_data.csv", index_col=0)

In [12]:
print(df_measuring_new.head())
print(df_offensive_new.head())
print(df_detecting_new.head())

                                               tweet  label
0  Yes indeed. She sort of reminds me of the elde...      0
1  The trans women reading this tweet right now i...      0
2  Question: These 4 broads who criticize America...      1
3  It is about time for all illegals to go back t...      0
4  For starters bend over the one in pink and kic...      1
                                               tweet  label
0  !!! RT @user: As a woman you shouldn't complai...      0
1  !!!!! RT @user: boy dats cold...tyga dwn bad f...      0
2  !!!!!!! RT @user Dawg!!!! RT @user: You ever f...      0
3   !!!!!!!!! RT @user: @user she look like a tranny      0
4  !!!!!!!!!!!!! RT @user: The shit you hear abou...      0
                                                                tweet  label
id                                                                          
572340447797903360  RT @user: Another bloody instant restaurant we...      0
560934139403436032  @user @user This video of the

In [13]:
df_new = pd.concat([df_measuring_new, df_offensive_new, df_detecting_new])

In [6]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176389 entries, 0 to 564921657240735744
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tweet   176389 non-null  object
 1   label   176389 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 4.0+ MB


In [7]:
df_new.label.value_counts()

0    123896
1     52493
Name: label, dtype: int64

In [8]:
df_new.head()

,tweet,label
0,Yes indeed. She sort of reminds me of the elde...,0
1,The trans women reading this tweet right now i...,0
2,Question: These 4 broads who criticize America...,1
3,It is about time for all illegals to go back t...,0
4,For starters bend over the one in pink and kic...,1


In [17]:
df_new = df_new.sample(frac=1).reset_index(drop=True)

In [18]:
df_new.to_csv("train_extension.csv")